# Predicting the Yellow Cab Drivers Pay

We fit a LinearRegression model to predict the yellow cab drivers pay.

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df = pd.read_csv('cleaned_taxi_data.csv')

In [3]:
df.head()

,Unnamed: 0,trip_distance,time_difference,drivers_pay,PU_hour
0,0,1.5,400.0,8.65,0
1,1,2.6,1152.0,15.00,0
2,7,1.3,429.0,7.75,0
3,8,3.7,818.0,17.20,0
4,9,2.1,720.0,11.70,0


In [4]:
X0 = df[['trip_distance','time_difference','drivers_pay','PU_hour']]

In [5]:
X0.head()

,trip_distance,time_difference,drivers_pay,PU_hour
0,1.5,400.0,8.65,0
1,2.6,1152.0,15.00,0
2,1.3,429.0,7.75,0
3,3.7,818.0,17.20,0
4,2.1,720.0,11.70,0


In [6]:
X0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7606168 entries, 0 to 7606167
Data columns (total 4 columns):
 #   Column           Dtype  
---  ------           -----  
 0   trip_distance    float64
 1   time_difference  float64
 2   drivers_pay      float64
 3   PU_hour          int64  
dtypes: float64(3), int64(1)
memory usage: 232.1 MB


We try to predict the *drivers_pay* using the *trip_distance* and the *time_difference*. *time_difference* is the time it took for the trip.

In [7]:
X = X0[['trip_distance','time_difference']]
Y = X0[['drivers_pay']]

In [8]:
X.head()

,trip_distance,time_difference
0,1.5,400.0
1,2.6,1152.0
2,1.3,429.0
3,3.7,818.0
4,2.1,720.0


In [9]:
Y.head()

,drivers_pay
0,8.65
1,15.00
2,7.75
3,17.20
4,11.70


In [10]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3, random_state = 1)

In [11]:
regr = LinearRegression()

In [12]:
regr.fit(X_train,Y_train)

LinearRegression()

In [13]:
Y_hat = regr.predict(X_test)

In [14]:
print('Coefficients: \n', regr.coef_)
print('Mean squared error: %.2f'
      % mean_squared_error(Y_test, Y_hat))
print('Coefficient of determination: %.2f'
      % r2_score(Y_test, Y_hat))

Coefficients: 
 [[3.05565197e+00 4.14695269e-05]]
Mean squared error: 170273.82
Coefficient of determination: 0.00


It is noticed that the Mean Squared Error is very high and the Coefficient of Determination is *0*. Which indicates that the *drivers_pay* is not predictable by the used predictors in general.

Therefore we take a different approach. We try to check whether there is a correlation when the data is considered hour by hour. The analysis is as follows.

In [15]:
def hourly_correlation(X_hour):
    X = X_hour[['trip_distance','time_difference']]
    Y = X_hour[['drivers_pay']]
    
    X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size = 0.3, random_state = 1)
    
    regr = LinearRegression()
    regr.fit(X_train,Y_train)
    
    Y_hat = regr.predict(X_test)
    
    return mean_squared_error(Y_test, Y_hat),r2_score(Y_test, Y_hat)

In [18]:
X1 = df[df.PU_hour == 0]
mse,r2 = hourly_correlation(X1)
print('Mean squared error: %.2f'
      % mse)
print('Coefficient of determination: %.2f'
      % r2)


Mean squared error: 24.37
Coefficient of determination: 0.86


In [20]:
mse_list = []
r2_list = []
hour = []

for i in range(24):
    X_hour = df[df.PU_hour == i]
    mse,r2 = hourly_correlation(X_hour)
    
    mse_list.append(mse)
    r2_list.append(r2)
    hour.append(i)

In [ ]:
print(hour)

## Analysis of the hourly Mean Squared Error

From the bar plot we can observe that the variance (MSE) of the **drivers pay** is exceptionally high during the 13th hour of the day. This may be due to the fact that people going out for lunch during this hour. This leads to increased demand which can be levered to ask for higher rates.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14,5))
_ = sns.barplot(hour , mse_list, ax = ax)
ax.set_xlabel('Pick Up Hour')
ax.set_ylabel('MSE of the prediction')
_ = ax.set_title('Mean Squared Error of prediction during the hour of the day')

### Analysis of the hourly Coffecient of Determination

Similar to the analysis of MSE we can see the same outlying behavior during *13th* hour in *r2_score* as well. During the all the other hours *r2_score* is high as well as uniformly distributed. We can fairly conclude that with the exception of the *13th* hour the pay rate can be predicted based on the *trip_distance* and the *trip_time* with a high accuracy.

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(14,5))
_ = sns.barplot(hour , r2_list, ax = ax)
ax.set_xlabel('Pick Up Hour')
ax.set_ylabel('r2_score of the prediction')
_ = ax.set_title('Coefficient of determination of prediction during the hour of the day')